## 仕入商品分析Ⅸ

- 分析（仕入先CD=90396）
- 使用不可

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from dateutil.relativedelta import relativedelta # 翌月の算出

!pip install japanize-matplotlib
import japanize_matplotlib

     |████████████████████████████████| 4.1 MB 5.0 MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120274 sha256=d56cd0dcf596f594425bb92b69a97e924527734b6e832bbb4211eb566bf77253
  Stored in directory: /root/.cache/pip/wheels/83/97/6b/e9e0cde099cc40f972b8dd23367308f7705ae06cd6d4714658
Successfully built japanize-matplotlib


In [ ]:
target_col = ['仕入先CD', '仕入先名', '仕入月', '品目', '品名', '規格', '大分類CD', '大分類名', '仕入数', '仕入単価', '仕入金額', '年度']
siiredf_n = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/N_S_2007_2021.csv", usecols=target_col)

target_col = ['年度', '売上月', '品目', '品名', '規格', '販売数量', '販売単価', '販売金額']
uriagedf_n = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/N_U_2019_2021.csv", usecols=target_col)

target_col = ['棚卸月', '品目', '品名', '規格', '在庫数', '在庫単価', '在庫金額']
zaikodf_n = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/N_Z_2019_2021.csv", usecols=target_col)

target_col = ['年度', '売上月', '品目', '品名', '規格', '販売数量', '販売単価', '販売金額']
uriagedf_i = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/I_U_2009_2021.csv", usecols=target_col)

target_col = ['棚卸月', '品目', '品名', '規格', '在庫数', '在庫単価', '在庫金額']
zaikodf_i = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/I_Z_2009_2021.csv", usecols=target_col)

u_s = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/U-S.csv", encoding='cp932')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
siiredf_n = siiredf_n[siiredf_n['仕入先CD'] == 90396]
siiredf_n['品目'] = siiredf_n['品目'].astype(int)
siiredf_n['仕入数'] = siiredf_n['仕入数'].astype(int)
siiredf_n['仕入金額'] = siiredf_n['仕入金額'].astype(int)
siiredf_n.head()

In [ ]:
# 仕入コード→製品コードのdictを作成
u_s = u_s.fillna(0)
u_s_dict = {}
chk_list = ['G', 'Z']
for i in range(len(u_s)):
    if u_s.iloc[i]['品目.1'] != 0:
        if u_s.iloc[i]['品目'][0] not in chk_list:
            if (int(u_s.iloc[i]['品目.1']), 0) in u_s_dict:
                j = 0
                k = 0
                while j == 0:
                    if (int(u_s.iloc[i]['品目.1']), k) in u_s_dict:
                        k += 1
                    else:
                        u_s_dict[int(u_s.iloc[i]['品目.1']), k] = int(u_s.iloc[i]['品目'])
                        j = k
            else:
                u_s_dict[int(u_s.iloc[i]['品目.1']), 0] = int(u_s.iloc[i]['品目'])

# u_sの'品目'にある品目CDがu_s_dictにあれば削除　
for i in range(len(u_s)):
    if u_s.iloc[i]['品目'][0] not in chk_list:
        if u_s.iloc[i]['品目.1'] == 0:
            j = 0
            k = 0
            while j == 0:
                if (int(u_s.iloc[i]['品目']), k) in u_s_dict:
                    del u_s_dict[int(u_s.iloc[i]['品目']), k]
                    k += 1
                else:
                    j =  1

In [ ]:
# Medit 製品品目設定
hinmoku_dict = {}
for i in range(len(siiredf_n)):
    j = 0
    k = 0
    while j == 0:
        if (siiredf_n.iloc[i]['品目'], k) in u_s_dict:
            j = 1
            if u_s_dict[siiredf_n.iloc[i]['品目'], k] not in hinmoku_dict:
                hinmoku_dict[siiredf_n.iloc[i]['品目']] = u_s_dict[siiredf_n.iloc[i]['品目'], k]
        else:
            k += 1
            j = k

medit_cd_list = []  # 製品CDlist
for i in range(len(siiredf_n)):
    if siiredf_n.iloc[i]['品目'] in hinmoku_dict:
        if hinmoku_dict[siiredf_n.iloc[i]['品目']] not in medit_cd_list:
            medit_cd_list.append(hinmoku_dict[siiredf_n.iloc[i]['品目']])
    else:
        if siiredf_n.iloc[i]['品目'] not in medit_cd_list:
            medit_cd_list.append(siiredf_n.iloc[i]['品目'])

medit_siire_list = [] # 仕入CDlist
for i in range(len(siiredf_n)):
    if siiredf_n.iloc[i]['品目'] not in medit_siire_list:
            medit_siire_list.append(siiredf_n.iloc[i]['品目'])

In [ ]:
medit_cd_list = [i for i in medit_cd_list if i < 500000]
medit_siire_list = [i for i in medit_siire_list if i < 900000]

In [ ]:
siiredf_n.head()

In [ ]:
siiredf_n['品名'].unique()

In [ ]:
siiredf_n['仕入月'] = pd.to_datetime((siiredf_n['仕入月']*100+1), format='%Y%m%d')

In [ ]:
tmp_df = siiredf_n.groupby('仕入月')['仕入金額'].sum()
idx = pd.to_datetime(tmp_df.index)
idx = np.array(idx)
values = tmp_df.values.astype(float) 
num = np.arange(len(idx))

p = np.polyfit(num, values, 1)
f = np.poly1d(p)
x_reg = num
y_reg = f(x_reg)

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(111, title='')
ax.plot(idx, values)
ax.plot(idx, y_reg, color='r', label='kaiki')
plt.show()

In [ ]:
siiredf_n.columns

Index(['仕入先CD', '仕入先名', '仕入月', '品目', '品名', '規格', '大分類CD', '大分類名', '仕入数',
       '仕入単価', '仕入金額', '年度'],
      dtype='object')

In [ ]:
tmp_df = siiredf_n[siiredf_n['年度']>2016]
                   
tmp_df = tmp_df.groupby('仕入月')['仕入金額'].sum()
idx = pd.to_datetime(tmp_df.index)
idx = np.array(idx)
values = tmp_df.values.astype(float) 
num = np.arange(len(idx))

p = np.polyfit(num, values, 1)
f = np.poly1d(p)
x_reg = num
y_reg = f(x_reg)

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(111, title='')
ax.plot(idx, values)
ax.plot(idx, y_reg, color='r', label='kaiki')
plt.show()

In [ ]:
tmp_df = siiredf_n[siiredf_n['年度']==2020]
tmp_df['仕入金額'].sum()

23703316

In [ ]:
tmp_df = tmp_df.groupby('品名')['仕入金額'].sum()
tmp_df = pd.DataFrame(tmp_df, columns=['仕入金額'])

In [ ]:
tmp_df['品名'] = tmp_df.index
tmp_df

In [ ]:
tmp_df

In [ ]:
# 売上データから製品を抽出
uriagedf_n = uriagedf_n[uriagedf_n['品目'].isin(medit_cd_list)]

In [ ]:
uriagedf_n['売上月'] = uriagedf_n['売上月'].astype(int)
uriagedf_n['品目'] = uriagedf_n['品目'].astype(int)
uriagedf_n['販売数量'] = uriagedf_n['販売数量'].astype(int)
uriagedf_n['販売単価'] = uriagedf_n['販売単価'].astype(int)
uriagedf_n['販売金額'] = uriagedf_n['販売金額'].astype(int)

In [ ]:
uriagedf_n['売上月'] = pd.to_datetime((uriagedf_n['売上月']*100+1), format='%Y%m%d')

In [ ]:
tmp=uriagedf_n['品目'].unique()
sorted(tmp)

In [ ]:
tmp_df = uriagedf_n.groupby('売上月')['販売金額'].sum()
idx = pd.to_datetime(tmp_df.index)
idx = np.array(idx)
values = tmp_df.values.astype(float) 
num = np.arange(len(idx))

p = np.polyfit(num, values, 1)
f = np.poly1d(p)
x_reg = num
y_reg = f(x_reg)

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(111, title='売上推移')
ax.plot(idx, values)
ax.plot(idx, y_reg, color='r', label='kaiki')
plt.show()

In [ ]:
# listを変換
mi_list = []
for i in medit_cd_list:
    mi_list.append("I" + str(i))

In [ ]:
# データから製品を抽出
uriagedf_i = uriagedf_i[uriagedf_i['品目'].isin(mi_list)]

In [ ]:
uriagedf_i['売上月'] = uriagedf_i['売上月'].astype(int)
uriagedf_i['販売数量'] = uriagedf_i['販売数量'].astype(int)
uriagedf_i['販売単価'] = uriagedf_i['販売単価'].astype(int)
uriagedf_i['販売金額'] = uriagedf_i['販売金額'].astype(int)

In [ ]:
uriagedf_i['売上月'] = pd.to_datetime((uriagedf_i['売上月']*100+1), format='%Y%m%d')

In [ ]:
tmp_df = uriagedf_i.groupby('売上月')['販売金額'].sum()
idx = pd.to_datetime(tmp_df.index)
idx = np.array(idx)
values = tmp_df.values.astype(float) 
num = np.arange(len(idx))

p = np.polyfit(num, values, 1)
f = np.poly1d(p)
x_reg = num
y_reg = f(x_reg)

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(111, title='売上推移')
ax.plot(idx, values)
ax.plot(idx, y_reg, color='r', label='kaiki')
plt.show()

In [ ]:
# 在庫データから製品を抽出
mn_list = m_list + m_siire_list
zaikodf_n = zaikodf_n[zaikodf_n['品目'].isin(mn_list)]

In [ ]:
zaikodf_n['棚卸月'] = zaikodf_n['棚卸月'].astype(int)
zaikodf_n['品目'] = zaikodf_n['品目'].astype(int)
zaikodf_n['在庫数'] = zaikodf_n['在庫数'].astype(int)
zaikodf_n['在庫単価'] = zaikodf_n['在庫単価'].astype(int)
zaikodf_n['在庫金額'] = zaikodf_n['在庫金額'].astype(int)

In [ ]:
zaikodf_n['棚卸月'] = pd.to_datetime((zaikodf_n['棚卸月']*100+1), format='%Y%m%d')

In [ ]:
tmp_df = zaikodf_n.groupby('棚卸月')['在庫金額'].sum()
idx = pd.to_datetime(tmp_df.index)
idx = np.array(idx)
values = tmp_df.values.astype(float) 
num = np.arange(len(idx))

p = np.polyfit(num, values, 1)
f = np.poly1d(p)
x_reg = num
y_reg = f(x_reg)

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(111, title='在庫推移')
ax.plot(idx, values)
ax.plot(idx, y_reg, color='r', label='kaiki')
plt.show()

In [ ]:
zaikodf_n[zaikodf_n['棚卸月']=="2021-03-01"]

In [ ]:
zaikodf_n

In [ ]:
# 在庫データから製品を抽出
zaikodf_i = zaikodf_i[zaikodf_i['品目'].isin(medit_icast_list)]

In [ ]:
zaikodf_i['棚卸月'] = zaikodf_i['棚卸月'].astype(int)
zaikodf_i['在庫数'] = zaikodf_i['在庫数'].astype(int)
zaikodf_i['在庫単価'] = zaikodf_i['在庫単価'].astype(int)
zaikodf_i['在庫金額'] = zaikodf_i['在庫金額'].astype(int)

In [ ]:
zaikodf_i['棚卸月'] = pd.to_datetime((zaikodf_i['棚卸月']*100+1).astype(str))

In [ ]:
tmp_df = zaikodf_i.groupby('棚卸月')['在庫金額'].sum()
idx = pd.to_datetime(tmp_df.index)
idx = np.array(idx)
values = tmp_df.values.astype(float) 
num = np.arange(len(idx))

p = np.polyfit(num, values, 1)
f = np.poly1d(p)
x_reg = num
y_reg = f(x_reg)

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(111, title='在庫推移')
ax.plot(idx, values)
ax.plot(idx, y_reg, color='r', label='kaiki')
plt.show()

In [ ]:
print(zaikodf_n.shape)
print(zaikodf_i.shape)

(167, 7)
(213, 7)


In [ ]:
zaikodf = pd.concat([zaikodf_n, zaikodf_i])

In [ ]:
tmp_df = zaikodf.groupby('棚卸月')['在庫金額'].sum()
idx = pd.to_datetime(tmp_df.index)
idx = np.array(idx)
values = tmp_df.values.astype(float) 
num = np.arange(len(idx))

p = np.polyfit(num, values, 1)
f = np.poly1d(p)
x_reg = num
y_reg = f(x_reg)

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(111, title='在庫推移')
ax.plot(idx, values)
ax.plot(idx, y_reg, color='r', label='kaiki')
plt.show()

In [ ]:
zaikodf[zaikodf['棚卸月']=="2021-03-01"]

### 粗利計算

In [ ]:
# 売上データセット
arari_n_dict = {}
for i in range(len(uriagedf_n)):
    tmp_cd = uriagedf_n.iloc[i]['品目']
    tmp_date = uriagedf_n.iloc[i]['売上月']
    if (tmp_cd, str(tmp_date)[:10]) in arari_n_dict:
        arari_n_dict[tmp_cd, str(tmp_date)[:10]][0] = uriagedf_n.iloc[i]['販売金額']
    else:
        arari_n_dict[tmp_cd, str(tmp_date)[:10]] = [uriagedf_n.iloc[i]['販売金額'], 0, 0, 0, 0, 0]

In [ ]:
# 在庫データセット(期末在庫)
for i in range(len(zaikodf_n)):
    tmp_cd = zaikodf_n.iloc[i]['品目']
    tmp_date = zaikodf_n.iloc[i]['棚卸月']
    if (tmp_cd, str(tmp_date)[:10]) in arari_n_dict:
        arari_n_dict[tmp_cd, str(tmp_date)[:10]][3] = zaikodf_n.iloc[i]['在庫金額']
    else:
        arari_n_dict[tmp_cd, str(tmp_date)[:10]] = [0, 0, 0, zaikodf_n.iloc[i]['在庫金額'], 0, 0]

In [ ]:
# 在庫データセット(期首在庫)
for i in range(len(zaikodf_n)):
    tmp_cd = zaikodf_n.iloc[i]['品目']
    tmp_date = zaikodf_n.iloc[i]['棚卸月']
    thismonth = datetime.datetime(today.year, today.month, 1)
    nextm = today + relativedelta(months=1)
    if (tmp_cd, str(nextm)[:10]) in arari_n_dict:
        arari_n_dict[tmp_cd, str(nextm)[:10]][1] = zaikodf_n.iloc[i]['在庫金額']
    else:
        arari_n_dict[tmp_cd, str(nextm)[:10]] = [0, zaikodf_n.iloc[i]['在庫金額'], 0, 0, 0, 0]